# Downloading a file from SharePoint site

### 3 Methods:

1. Using [spLite](https://github.com/mstackhouse/spLite) to save file to local drive or as pandas dataframe (in-memory)
2. Using shututil, requests+requests_ntlm to save file to local drive
3. Using io, pandas, requests+requests_ntlm to save file as pandas dataframe (in-memory)

Example assumes SharePoint security access is based on [NTLM](https://en.wikipedia.org/wiki/Integrated_Windows_Authentication) Integrated Windows Authentication)

In [ ]:
from getpass import getpass
from requests_ntlm import HttpNtlmAuth
from spLite import sharepoint
import io
import os
import pandas as pd
import requests
import shutil
import sys

In [ ]:
user = os.environ['windows_user']
pwd = os.environ['windows_pwd']

In [ ]:
DOMAIN = 'YOUR_DOMAIN'

In [ ]:
#Enter your SharePoint site (without default.html)
sharePointUrl = 'http://hondateamlink.ham.am.honda.com/sites/PHLogistics'
folderUrl = 'Shared Documents'

In [ ]:
s = sharepoint.SpSession(sharePointUrl, f'{DOMAIN}\\{user}', pwd, context_type='ntlm')

### Method 1: Download file and saving to local drive

In [ ]:
s.get_file(folderUrl, "some_excel_file.xlsx", output_location='D:\\temp\\')

or

download file and save as a pandas dataframe (in-memory)

In [ ]:
df = pd.read_excel(s.get_file(folderUrl, "your_excel_file.xlsx"))

### Method 2: Using requests+requests_ntlm to save file to local drive

In [ ]:
file = 'http://some_sharepoint_site.com/sites/Some_Dept/Shared%20Documents/your_excel_file.xlsx'

r = requests.get(file, auth=HttpNtlmAuth(f'{DOMAIN}\\{user}', pwd), stream=True)
if r.status_code == 200:
    with open(r'D:\temp\window_master.xlsx', 'wb') as f:
        r.raw.decode_content = True
        shutil.copyfileobj(r.raw, f)
    print("Completed downloading file")
else:
    print("Problem with accessing site")

### Method 3: Using io, pandas, requests+requests_ntlm to save file as pandas dataframe (in-memory)

In [ ]:
import pandas as pd
import io

r = requests.get(file, auth=HttpNtlmAuth(f'{DOMAIN}\\{user}', pwd), stream=True)
with io.BytesIO(r.content) as fh:
    df = pd.io.excel.read_excel(fh, sheet_name='WM', skiprows=6)